# Image Augmentation
### It takes user input for the total number of images to be produced(Augmented images and the original images). 
### Also added blending feature in augmentation.

In [1]:
import os
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.parameters import Choice
import numpy as np

C:\Users\rishi\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\rishi\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\rishi\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


### Enter the paths for Augmentation

In [6]:
# Path to the directory containing the images
base_directory = r"D:\Experiment_Dataset\"

# Output directory to store augmented images
output_directory = r"D:\Experiment_Dataset\Augmented_Dataset_5_Regions\val"
os.makedirs(output_directory, exist_ok=True)

In [7]:
# Function to get user input for the desired number of augmented images
def get_desired_num_images():
    while True:
        try:
            num_images = int(input("Enter the desired number of augmented images per class: "))
            if num_images <= 0:
                print("Please enter a positive integer.")
                continue
            else:
                return num_images
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

# Get a list of class (folder) names
class_names = [name for name in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, name))]

# Define augmentations
augmentations = [
    iaa.ScaleX((0.5, 1.5)),
    iaa.Affine(scale=(0.5, 1.5)),
    iaa.Affine(rotate=(-25, 25)),
    iaa.AdditiveGaussianNoise(scale=(10, 60)),
    iaa.Crop(percent=(0, 0.2)),
    iaa.PiecewiseAffine(scale=(0.01, 0.05)),
    iaa.ElasticTransformation(alpha=(0, 5.0), sigma=0.25),
    iaa.Rot90(1),
    iaa.BlendAlpha(
        factor=(0.2, 0.8),
        foreground=iaa.Affine(rotate=(-20, 20)),
        per_channel=True
    ),
    iaa.BlendAlphaSimplexNoise(
        foreground=iaa.Multiply(Choice([0.5, 1.5]), per_channel=True)
    )
]



# Get user input for the desired number of augmented images per class
desired_num_images = get_desired_num_images()

# Calculate the maximum number of images in any class
max_images_count = max([len(os.listdir(os.path.join(base_directory, class_name))) for class_name in class_names])

# Calculate the augmentation factor for each class
augmentation_factors = {}
for class_name in class_names:
    directory_path = os.path.join(base_directory, class_name)
    num_images = len([file for file in os.listdir(directory_path) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
    augmentation_factors[class_name] = desired_num_images / num_images

# Iterate over each class (folder)
for class_name in class_names:
    # Construct the full path to the directory containing images for this class
    directory_path = os.path.join(base_directory, class_name)
    
    # Get a list of files (images) in the class directory
    image_files = [file for file in os.listdir(directory_path) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    
    # Create a directory to store augmented images for this class
    class_output_directory = os.path.join(output_directory, class_name)
    os.makedirs(class_output_directory, exist_ok=True)
    
    # Iterate over each image file in the class directory
    for filename in image_files:
        # Construct the full path to the image file
        image_path = os.path.join(directory_path, filename)

        # Load the image using imageio
        image = imageio.imread(image_path)

        # Determine the number of augmentations to apply to this image
        num_augmentations = round(augmentation_factors[class_name])

        # Perform and save each augmentation individually
        for i in range(num_augmentations):
            selected_augmentation = augmentations[i % len(augmentations)]
            augmented_image = selected_augmentation(image=image)
            output_filename = os.path.splitext(filename)[0] + f"_aug{i}" + os.path.splitext(filename)[1]
            output_path = os.path.join(class_output_directory, output_filename)
            imageio.imwrite(output_path, augmented_image)

# Print the number of images before and after augmentation for each class
for class_name in class_names:
    directory_path = os.path.join(base_directory, class_name)
    num_images_before = len([file for file in os.listdir(directory_path) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
    directory_path = os.path.join(output_directory, class_name)
    num_images_after = len([file for file in os.listdir(directory_path) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
    print(f"Class: {class_name}")
    print(f"  Before augmentation: {num_images_before} images")
    print(f"  After augmentation: {num_images_after} images")
    print()

print("Augmentation completed successfully!")

Enter the desired number of augmented images per class: 428


C:\Users\rishi\AppData\Local\Temp\ipykernel_18036\1680201662.py:70: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


Class: Central
  Before augmentation: 16 images
  After augmentation: 432 images

Class: East
  Before augmentation: 10 images
  After augmentation: 430 images

Class: North
  Before augmentation: 69 images
  After augmentation: 414 images

Class: South
  Before augmentation: 101 images
  After augmentation: 404 images

Class: West
  Before augmentation: 87 images
  After augmentation: 435 images

Augmentation completed successfully!


### Enter the path to store after splitting

In [9]:
# # Define input folder path containing class folders with images
# # The output_directory is the directory made after augmentation.
# input_folder = output_directory

# # Define output folder path for the split dataset
# output_folder = 'D:\Augmented_Cattle_Breeds_Update_split'

In [10]:
# import os
# import shutil
# from sklearn.model_selection import train_test_split


# # Create output folder if it doesn't exist
# os.makedirs(output_folder, exist_ok=True)

# # Get list of class folders
# class_folders = [folder for folder in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, folder))]

# # Define train, test, and validation ratios
# train_ratio = 0.7
# test_ratio = 0.15
# val_ratio = 0.15

# # Iterate over each class folder
# for class_folder in class_folders:
#     # Construct the full path to the class folder
#     class_folder_path = os.path.join(input_folder, class_folder)
    
#     # Get list of image files in the class folder
#     image_files = [f for f in os.listdir(class_folder_path) if os.path.isfile(os.path.join(class_folder_path, f))]
    
#     # Skip splitting if there is only one sample
#     if len(image_files) <= 1:
#         print(f"Skipping splitting for class {class_folder} as it contains only one sample.")
#         continue
    
#     # Split image files into train, test, and validation sets
#     train_files, test_val_files = train_test_split(image_files, test_size=(test_ratio + val_ratio), random_state=1234)
#     test_files, val_files = train_test_split(test_val_files, test_size=(val_ratio / (test_ratio + val_ratio)), random_state=1234)
    
#     # Create corresponding folders in the output directory for train, test, and validation
#     train_class_folder = os.path.join(output_folder, 'train', class_folder)
#     test_class_folder = os.path.join(output_folder, 'test', class_folder)
#     val_class_folder = os.path.join(output_folder, 'val', class_folder)
    
#     os.makedirs(train_class_folder, exist_ok=True)
#     os.makedirs(test_class_folder, exist_ok=True)
#     os.makedirs(val_class_folder, exist_ok=True)
    
#     # Function to copy images from source to destination folder
#     def copy_images(source_files, destination_folder):
#         for file in source_files:
#             src = os.path.join(class_folder_path, file)
#             dst = os.path.join(destination_folder, file)
#             shutil.copyfile(src, dst)
    
#     # Copy images to train, test, and validation folders
#     copy_images(train_files, train_class_folder)
#     copy_images(test_files, test_class_folder)
#     copy_images(val_files, val_class_folder)

# print("Splitting completed successfully!")

Splitting completed successfully!
